<a href="https://colab.research.google.com/github/hegdenischay/NEURONNet/blob/main/Full%20Network/Latest_%20Complete%20Network_sensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
size =   1000#@param {type:"integer"}
dimension = "High" #@param ["High", "Low"]
epochs =  20 #@param {type:"integer"}
A_plus = 0.01 #@param {type:"number"}
A_minus = -0.0011 #@param {type: "number"}
lateralDelay = 5 #@param {type: "number"}

In [2]:
#@title Parameters
from ipywidgets import FloatText, Checkbox, VBox, Dropdown
import ipywidgets as widgets

currentAmp = FloatText(description='Amplitude:', value=0.1)
sets = FloatText(description='sets:', value=15)
noiseSD = FloatText(description='noiseSD:', value=0.01)

vb = VBox(children = [currentAmp, sets])
with_noise = Checkbox(description='Add noise')

def add_noise(button):
    if button['new']:
        vb.children = [currentAmp, sets, noiseSD]
    else:
        vb.children = [currentAmp, sets]

with_noise.observe(add_noise, names='value')
display(with_noise)
display(vb)

Checkbox(value=False, description='Add noise')

In [3]:
#@title Distribution
from ipywidgets import FloatText, Checkbox, VBox, Dropdown
import ipywidgets as widgets

distribution = Dropdown(description='distribution:', options=["uniform", "normal", "poisson"], value="uniform")
low = FloatText(description='low:', value=1)
high = FloatText(description='high:', value=1.5)
leng = FloatText(description='size:', value=sets.value)
mean = FloatText(description='mean:', value=1)
sd = FloatText(description='SD:', value=0.5)
lam = FloatText(description='lam', value=3)

vb2 = VBox(children = [low, high, leng])

def change_widgets(button):
    if button['new'] == "uniform":
        vb2.children = [low, high, leng]
    elif button['new'] == "normal":
        vb2.children = [mean, sd, leng]
    elif button['new'] == "poisson":
        vb2.children = [lam, sets]

distribution.observe(change_widgets, names='value')
display(distribution)
display(vb2)

Dropdown(description='distribution:', options=('uniform', 'normal', 'poisson'), value='uniform')

### Install Neuron

In [4]:
pip install pandas neuron wget

     |████████████████████████████████| 15.0 MB 4.8 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=815d7789ce10fa5550c874e731b83682a7884a4ed5f8f75f0f90ac9ce9fc53d0
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [5]:
# %load_ext google.colab.data_table
# import pandas
# data = {'Time taken': ['5 sets (with learning)', '5 sets (without learning)', '30 sets (with learning)', '30 sets (without)'], 'Time (s)': [64.94386959075928, 64.94386959075928-47.318169593811035, 163.1467843055725, 163.1467843055725-123.22148942947388]}  
# pandas.DataFrame(data)

### Import prerequisites

In [6]:
import time
totalstart = time.time()
import os
if os.getcwd() != "/content":
    # we are not in google colab, assume hebbian library is in current dir
    from hebb import ORN, MCELL, GCELL
else:
    # download from git
    import shutil, requests
    url = 'https://cloud.operationtulip.com/s/t8nbn8Y582w94P3/download/git.zip'
    response = requests.get(url, stream=True)
    with open('git.zip', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response
    import zipfile
    with zipfile.ZipFile('git.zip', 'r') as zip_ref:
        zip_ref.extractall('')


In [7]:
!nrnivmodl &>/dev/null

In [8]:
from neuron import h #.h is a HOC object instance & gui from neuron can also be imported 
from math import pi                  
from neuron.units import ms,mV
from hebb_test import ORN, MCELL, GCELL
h.load_file('stdrun.hoc')                #Allows us to do a high level simulation
pc = h.ParallelContext()
import random  
import matplotlib.pyplot as plt1

from bokeh.io import output_notebook
import bokeh.plotting as plt2
output_notebook()

import seaborn as sns

## Helper Functions

In [9]:
def stairstep(lo: float, hi: float, duration: int, max_duration: int = 200, noise: list = []) -> list:
    # duration and max_duration are in ms
    ans = [lo if i < duration else hi for i in range(0,max_duration)]
    if len(noise) == 0:
        return ans
    else:
        assert len(ans) == len(noise)
        return [ans[i]+noise[i] for i in range(len(ans))]
    # return [0+0.8*(i/duration) if i < duration else 0 for i in range(0,max_duration)]
    # visualize input 

In [10]:
def netConMaker(source, dest, weights, delay, threshold, place='axon', type='exc'):
    # print(source, dest)
    if place == 'axon':
        if type == 'exc':
            netcon = h.NetCon(source.axon(0.5)._ref_v, dest.dendexcisyn, sec=source.axon)
        elif type == 'inh':
            netcon = h.NetCon(source.axon(0.5)._ref_v, dest.dendinhisyn, sec=source.axon)
    elif place == 'soma':
        if type == 'exc':
            netcon = h.NetCon(source.soma(0.5)._ref_v, dest.dendexcisyn, sec=source.soma)
        elif type == 'inh':
            netcon = h.NetCon(source.soma(0.5)._ref_v, dest.dendinhisyn, sec=source.soma)
    netcon.weight[0] = weights
    netcon.delay = delay
    netcon.threshold = threshold
    return netcon

In [11]:
def latConMaker(source, dest, weights, delay, threshold, type='exc'):
    if type == 'exc':
        netcon = h.NetCon(source.dend(0.5)._ref_v, dest.dendexcisyn, sec=source.dend)
    else:
        netcon = h.NetCon(source.dend(0.5)._ref_v, dest.dendinhisyn, sec=source.dend)
    netcon.weight[0] = weights
    netcon.delay = delay
    netcon.threshold = threshold
    return netcon

In [12]:
%matplotlib inline
def plotCell(source, name, addendum=''):
    recording_cell = source
    cell_axon = h.Vector().record(recording_cell.axon(0.5)._ref_v)
    cell_dend = h.Vector().record(recording_cell.dend(0.5)._ref_v)
    t = h.Vector().record(h._ref_t)

    h.finitialize(-70 * mV)
    h.continuerun(300 * ms)

    f = plt1.figure(figsize=(16,9))
    ax = f.add_subplot(1,1,1)
    ax.plot(t, list(cell_axon), label=name+' axon '+addendum)
    ax.plot(t, list(cell_dend), label=name+' dendrite '+addendum)
    ax.legend(loc="upper right")
    ax.set_xlabel('t (ms)')
    ax.set_ylabel('v (mV)')
    plt1.show()

In [13]:
import math
def generate_L_weight_delta(first, second):
    A_plus = 0.01
    A_minus = -0.011
    tau_pre =20*ms
    tau_post =20*ms
    delta_t = [(second[iter] - first[iter]) for iter in range(min(len(first),len(second)))]
    delta_w_list = [A_plus*math.exp(-delta_t[iter]/tau_post) if delta_t[iter] >=0 else A_minus*math.exp(delta_t[iter]/tau_pre) for iter in range(min(len(first),len(second)))]  
    delta_w = sum(delta_w_list)
    return delta_w

In [14]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

## Network Definitions

In [15]:
class AON:
    def __init__(self,M,weights,delay1=1,delay2=2,delay3=3,delay4=4):
        wMM=weights[0]
        wMG=weights[1]
        wGM=weights[2]
        wGG=weights[3]
        
        self.M = M #Set no.
        self.th = -70
        self.maindelay = 5

        self.M1=MCELL(1,self.M)
        self.M2=MCELL(2,self.M)
        self.G1=GCELL(3,self.M)
        self.G2=GCELL(4,self.M)
        self.cells = [self.M1, self.M2, self.G1, self.G2]
        
        self.nc0 = netConMaker(self.M1, self.M2, wMM, delay1+self.maindelay, self.th, 'soma') #Connecting E1 to E2
        self.nc1 = netConMaker(self.M2, self.M1, wMM, delay2+self.maindelay, self.th, 'soma') #Connecting E2 to E1
        self.nc2 = netConMaker(self.M2, self.G1, wMG, delay2+self.maindelay, self.th, 'soma') #Connecting E2 to I1
        self.nc3 = netConMaker(self.G1, self.M2, wGM, delay3+self.maindelay, self.th, 'soma', type='inh') #Connecting I1 to E2
        self.nc4 = netConMaker(self.G1, self.G2, wGG, delay3+self.maindelay, self.th, 'soma', type='inh') #Connecting I1 to I2
        self.nc5 = netConMaker(self.G2, self.G1, wGG, delay4+self.maindelay, self.th, 'soma', type='inh') #Connecting I2 to I1
        self.nc6 = netConMaker(self.G2, self.M1, wGM, delay4+self.maindelay, self.th, 'soma', type='inh') #Connecting I2 to E1
        self.nc7 = netConMaker(self.M1, self.G2, wMG, delay1+self.maindelay, self.th, 'soma') #Connecting E1 to I2
        self.nc8 = netConMaker(self.M1, self.G1, wMG, delay1+self.maindelay, self.th, 'soma') #Connecting E1 to I1
        self.nc9 = netConMaker(self.G1, self.M1, wGM, delay3+self.maindelay, self.th, 'soma', type='inh') #Connecting I1 to E1

In [18]:
import wget
import pandas as pd
file = wget.download('https://cloud.operationtulip.com/s/ySazp9r54BZnDZc/download/dataset%28in_CSV%29.csv')
dataframe1 = pd.read_csv(file)
inp = []
inp.append([dataframe1["GAS1"][i] for i in range(len(dataframe1["GAS1"])) if dataframe1["label"][i] == "Banana"])
inp.append([dataframe1["GAS2"][i] for i in range(len(dataframe1["GAS2"])) if dataframe1["label"][i] == "Banana"])
inp.append([dataframe1["GAS3"][i] for i in range(len(dataframe1["GAS3"])) if dataframe1["label"][i] == "Banana"])

In [19]:
#Gives one 2 coupled neuron using freeman's topology in olfactory bulb
class CONNECTING:

    def __init__(self,M,weights,delay1=0,delay2=2,delay3=4,delay4=6):
        wMM=weights[0]
        wMG=weights[1]
        wGM=weights[2]
        wGG=weights[3]
        
        self.M = M #Set no.
        self.th = -70
        self.maindelay = 5
        
        #Making 2 neurons
        self.orn = ORN(1,self.M)
        self.P1=MCELL(2,self.M)
        self.P2=MCELL(3,self.M)
        self.cells = [self.orn, self.P1, self.P2]
        
        # #Giving current pulse
        self.stim = h.VClamp(self.orn.dend(0.5))
        self.stim.delay = 10
        self.stim.dur = 100
        self.stim.amp = 0
        self.input = h.Vector(inp[M-1])
        self.tv = h.Vector([i for i in range(len(inp[M-1]))])
        self.input.play(self.stim._ref_v, self.tv, True)
        # self.stim = h.IClamp(self.orn.dend(0.5))  #P1.soma to P1.dend
        # # self.stimArr = [h.IClamp(self.orn.ciliumArr[i](0.5)) for i in range(len(self.orn.ciliumArr))]
        # if with_noise.value == True:
        #     self.noise = np.random.normal(0, noiseSD.value, 100)
        #     self.input = h.Vector(stairstep(0,currentAmp.value,10,100,self.noise))
        #     self.tv = h.Vector([i for i in range(100)])
        #     self.stim.delay = 10
        #     self.stim.dur = 100
        #     self.stim.amp = 0
        #     self.input.play(self.stim._ref_amp, self.tv, True) 
        # else:
        #     self.stim.delay = 10
        #     self.stim.dur = 100
        #     self.stim.amp = currentAmp.value
        
        self.nc0 = netConMaker(self.orn, self.P1, wMM, self.maindelay+delay2, self.th, 'soma') #Connecting ORN to P1
        self.nc1 = netConMaker(self.P1, self.P2, wMM, self.maindelay+delay3, self.th, 'soma') #Connecting P1 to P2
        self.nc2 = netConMaker(self.P2, self.P1, wMM, self.maindelay+delay3, self.th, 'soma') #Connecting P2 to P1
    
        # 4-coupled

        self.M1=MCELL(1,self.M)
        self.M2=MCELL(2,self.M)
        self.G1=GCELL(3,self.M)
        self.G2=GCELL(4,self.M)
        self.cells.extend([self.M1, self.M2, self.G1, self.G2])

        self.nc14 = netConMaker(self.orn, self.M1, wMM, delay1+self.maindelay, self.th, 'soma')
        self.nc3 = netConMaker(self.P1, self.M1, wMM, delay1+self.maindelay, self.th, 'soma')
        self.nc4 = netConMaker(self.M1, self.M2, wMM, delay2+self.maindelay, self.th, 'soma') #Connecting M1 to M2
        self.nc5 = netConMaker(self.M2, self.M1, wMM, delay2+self.maindelay, self.th, 'soma') #Connecting M2 to M1
        self.nc6 = netConMaker(self.M2, self.G1, wMG, delay2+self.maindelay, self.th, 'soma') #Connecting M2 to G1
        self.nc7 = netConMaker(self.G1, self.M2, wGM, delay2+self.maindelay, self.th, 'soma', type='inh') #Connecting G1 to M2
        self.nc8 = netConMaker(self.G1, self.G2, wGG, delay3+self.maindelay, self.th, 'soma', type='inh') #Connecting G1 to G2
        self.nc9 = netConMaker(self.G2, self.G1, wGG, delay3+self.maindelay, self.th, 'soma', type='inh') #Connecting G2 to G1
        self.nc10 = netConMaker(self.G2, self.M1, wGM, delay4+self.maindelay, self.th, 'soma', type='inh') #Connecting G2 to M1
        self.nc11 = netConMaker(self.M1, self.G2, wMG, delay4+self.maindelay, self.th, 'soma') #Connecting M1 to G2
        self.nc12 = netConMaker(self.M1, self.G1, wMG, delay1+self.maindelay, self.th, 'soma') #Connecting M1 to G1
        self.nc13 = netConMaker(self.G1, self.M1, wGM, delay1+self.maindelay, self.th, 'soma', type='inh') #Connecting G1 to M1

In [20]:

class LATERAL:
    """Laterally coupling N-2 coupled neurons
    """
    def __init__(self,N,weights,delay1=0,delay2=2,delay3=4,delay4=6):
        wMML=weights[1]
        wGGL=weights[5]
        
        self.N = N
        self.th = -70
        self.maindelay = lateralDelay
        self.sets = [CONNECTING(i, weights) for i in range(N)]
            
        self.twoCupArr = []
        for r in range(0,N-1):
          #Connecting P1[0] to P1[1]
          # make a list for easier access
          netConList = []
          netConList.append(latConMaker(self.sets[r].P1, self.sets[r+1].P1,wMML,self.maindelay+delay2,self.th)) #P1[0] to P1[1]
          netConList.append(latConMaker(self.sets[r+1].P1, self.sets[r].P1, wMML, self.maindelay+delay2,self.th))

          self.twoCupArr.append(netConList)
        self.fourCupArr = []
        for r in range(1,N-1):
            for j in range(N-r):
                netConList = []
                netConList.append(latConMaker(self.sets[j].M1, self.sets[j+r].M1, wMML, self.maindelay+delay3, self.th)) #Connect M1[0] to M1[1]
                netConList.append(latConMaker(self.sets[j+r].M1, self.sets[j].M1, wMML, self.maindelay+delay3, self.th)) #Connect M1[1] to M1[0]
                netConList.append(latConMaker(self.sets[j].G1, self.sets[j+r].G1, wGGL, self.maindelay+delay2, self.th, 'inh')) #Connecting G1[0] to G1[1]
                netConList.append(latConMaker(self.sets[j+r].G1, self.sets[j].G1, wGGL, self.maindelay+delay2, self.th, 'inh')) #Connecting G1[1] to G1[0]
                self.fourCupArr.append(netConList)

        # add AON and PC
        self.AON = AON(5,weights)
        self.PC = AON(6,weights)
        self.DPC = GCELL(7,self.N)
        # add noise to AON's M1
        self.stim = h.IClamp(self.AON.M1.dend(0.5))  #P1.soma to P1.dend
        self.noise = np.random.normal(0, 0.1, 100)
        self.input = h.Vector(self.noise)
        self.tv = h.Vector([i for i in range(100)])
        self.input.play(self.stim._ref_amp, self.tv, True)
        self.otherNetCons = []
        self.otherNetCons.append(netConMaker(self.sets[0].M1, self.AON.M1, wMML, self.maindelay+delay2, self.th, 'soma')) # 4-coupled M1 -> AON's E1
        for i in self.sets:
            for j in self.sets:
                if i != j:
                    self.otherNetCons.append(netConMaker(i.P1, j.P1, wMML, self.maindelay+delay2, self.th, 'soma')) # all-to-all in P1
                    self.otherNetCons.append(netConMaker(i.M1, j.M1, wMML, self.maindelay+delay3, self.th, 'soma')) # all-to-all in M1
                    self.otherNetCons.append(netConMaker(i.G1, j.G1, wGGL, self.maindelay+delay2, self.th, 'soma')) # all-to-all in M1

        for i in self.sets:
            self.otherNetCons.append(netConMaker(self.AON.M1, i.P2, wMML, self.maindelay+delay2, self.th, 'soma')) # AON E1 -> 2-coupled P2
        if dimension == "High":
            self.otherNetCons.append(netConMaker(self.PC.M1, self.AON.G1, wMML, self.maindelay+delay3+10, self.th, 'soma')) # PC's A1 to AON's I1
        self.otherNetCons.append(netConMaker(self.sets[0].M1, self.PC.M1, wMML, self.maindelay+delay2, self.th, 'soma')) # M1 to A1
        for i in self.sets:
            self.otherNetCons.append(netConMaker(self.AON.M1, i.G1, wMML, self.maindelay+delay2, self.th, 'soma')) # AON's E1 -> 4-coupled G1
        
        self.otherNetCons.append(netConMaker(self.PC.G1, self.DPC, wMML/100000, self.maindelay+delay3, self.th, 'soma', 'inh')) # PC's B1 -> DPC
        self.otherNetCons.append(netConMaker(self.DPC, self.PC.G1, wMML/100000, self.maindelay+delay3, self.th, 'soma')) # DPC -> PC's B1
        if dimension == "High":
            for i in self.sets:
                self.otherNetCons.append(netConMaker(self.DPC, i.G1, wMML/100000, self.maindelay+delay3+10, self.th))# DPC to G1 
        self.otherNetCons.append(netConMaker(self.sets[-1].G1, self.DPC, wMML, self.maindelay+4, self.th, 'soma'))
        


In [21]:
import numpy as np
if distribution.value == "uniform":
    weights = np.random.uniform(low.value, high.value, 2*int(leng.value)-1)
elif distribution.value == "normal":
    weights = np.random.normal(mean.value, sd.value, 2*int(leng.value)-1)
elif distribution.value == "poisson":
    weights = np.random.poisson(lam.value, 2*int(leng.value)-1)

In [22]:
#Give N value to get N no. of 2 coupled neurons laterally connected
#given_input_from_user = int(input('Enter the number of 2 coupled sets required: '))
import numpy as np
# size = 5
low, high = 1, 5
# all_weights = np.random.uniform(low, high,2*given_input_from_user-1)
# all_weights = [4.65058555, 2.92840154, 3.68134116, 1.30487304, 1.07771946, 3.15136447, 1.04247865, 2.29878494, 2.01195344]
# all_weights = [0.95336892, 0.87793015, 0.71060386, 0.88298338, 0.84561174, 0.5006465, 0.60983793, 0.88890724, 0.85189888]
L1=LATERAL(size,weights)

LookupError: ignored

# Show Topology

In [ ]:
h.topology()

In [ ]:
# # from neuron import h, gui
# from matplotlib import pyplot as plt
# import numpy
# for sec in h.allsec():
#     sec.nseg = 1
#     for seg in sec:
#         seg.diam = numpy.interp(seg.x, [0, 1], [10, 40])

# s = h.Shape()
# s.show(1)
# # s.color(2, sec=a) # color section "a" red
# # h.topology()
# h.finitialize(-65)
        
# fig = plt.figure()
# fig.set_figwidth(16)
# fig.set_figheight(9)
# ax = fig.add_subplot(projection='3d')

# MCell=[]
# GCell=[]
# ORNCell=[]
# ORNCellPlot=[[],[],[]]
# MCellPlot=[[],[],[]]
# GCellPlot=[[],[],[]]
# h.finitialize()
# for sec in h.allsec():
#     if 'ORNcell' in repr(sec):
#         ORNCell.append(sec)
#     elif 'Mcell' in repr(sec):
#         MCell.append(sec)
#     elif 'Gcell' in repr(sec):
#         GCell.append(sec)

# for sec in ORNCell:
#     for i in range(sec.n3d()):
#         ORNCellPlot[0].append(sec.x3d(i))
#         ORNCellPlot[1].append(sec.y3d(i))
#         ORNCellPlot[2].append(sec.z3d(i))
# for sec in MCell:
#     for i in range(sec.n3d()):
#         MCellPlot[0].append(sec.x3d(i))
#         MCellPlot[1].append(sec.y3d(i))
#         MCellPlot[2].append(sec.z3d(i))
# for sec in GCell:
#     for i in range(sec.n3d()):
#         GCellPlot[0].append(sec.x3d(i))
#         GCellPlot[1].append(sec.y3d(i))
#         GCellPlot[2].append(sec.z3d(i))
# #         print(sec.x3d(i), sec.y3d(i), sec.z3d(i))
# # print(ORNCellPlot, MCellPlot,GCellPlot)
# ax.scatter(ORNCellPlot[0], ORNCellPlot[1], ORNCellPlot[2], label='ORN')
# ax.scatter(MCellPlot[0], MCellPlot[1], MCellPlot[2], label='MCell')
# ax.scatter(GCellPlot[0], GCellPlot[1], GCellPlot[2], label='GCell')

# ax.set_xlabel('X coord')
# ax.set_ylabel('Y coord')
# ax.set_zlabel('Z coord')
# ax.legend()

# plt.show()

In [ ]:
# from neuron import h, gui2
# gui2.set_backend('jupyter')
# ps = gui2.PlotShape()
# ps.variable('v')
# ps.show(0)

In [ ]:
import math
def drawline(pre, post, ax):
#     ps.beginline()
#     print("Pre:", pre.axon.x3d(1), pre.axon.y3d(1), pre.axon.z3d(1))
#     print("Post:", post.axon.x3d(1), post.axon.y3d(1), post.axon.z3d(1))

    pre_x, pre_y, pre_z = pre.axon.x3d(0), pre.axon.y3d(0), pre.axon.z3d(0)
    post_x, post_y, post_z = post.axon.x3d(1), post.axon.y3d(1), post.axon.z3d(1)
#     print("pre_x", pre_x, post_x,"pre_y", pre_y, post_y,"pre_z", pre_z, post_z)
    ax.plot([pre_x, post_x], [pre_y, post_y], [pre_z, post_z])

In [ ]:
%matplotlib notebook
from neuron import h
from matplotlib import pyplot
ps = h.PlotShape(False)  # False tells h.PlotShape not to use NEURON's gui
ps.plot(pyplot)
fig = pyplot.figure(1)
ax = fig.axes[0]
print("Two-coupled")
for i in L1.twoCupArr:
    # get into netconList
    for j in i:
        # get P1 and P2
        drawline(j.precell(), j.postcell(), ax)
print('Four-coupled:')
for i in L1.fourCupArr:
    # get into netConList
    for j in i:
        drawline(j.precell(), j.postcell(),ax)


pyplot.show()
# ps.show()

# Voltages before Learning

In [ ]:
%matplotlib inline

In [ ]:
plotCell(L1.DPC, 'DPC')

In [ ]:
plotCell(L1.sets[0].orn, 'ORN')

In [ ]:
plotCell(L1.sets[0].M1, 'M1', '(4-coupled)')

In [ ]:
plotCell(L1.sets[0].M2, 'M2', '(4-coupled)')

In [ ]:
plotCell(L1.sets[0].G1, 'G1', '(4-coupled)')

In [ ]:
plotCell(L1.sets[0].G2, 'G2', '(4-coupled)')

In [ ]:
plotCell(L1.sets[0].P1, 'P1', '(2-coupled)')

In [ ]:
plotCell(L1.sets[0].P2, 'P2', '(2-coupled)')

# Voltage vs Voltage analysis

In [ ]:
%matplotlib inline
recording_cell = L1.sets[0].P1
recording_cell_2 = L1.sets[0].P2
axon_m1 = h.Vector().record(recording_cell.axon(0.5)._ref_v)
dend_m1 = h.Vector().record(recording_cell.dend(0.5)._ref_v)
axon_m2 = h.Vector().record(recording_cell_2.axon(0.5)._ref_v)
dend_m2 = h.Vector().record(recording_cell_2.dend(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)

# f = plt2.figure(x_axis_label='v (mV)', y_axis_label='v (mV)')
# f.line(list(axon_m1), list(axon_m2), line_width=1,legend_label='P1 axon vs P2 axon (2-coupled)',line_color='black')
# f2 = plt2.figure(x_axis_label='v (mV)', y_axis_label='v (mV)')
# f2.line(list(dend_m1), list(dend_m2), line_width=2,legend_label='P1 dendrite vs P2 dendrite (2-coupled)',line_color='black')
# plt2.show(f)
# plt2.show(f2)

fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(axon_m1), list(axon_m2), label='P1 axon vs P2 axon (2-coupled)')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('t (ms)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(dend_m1),list(dend_m2), label='P1 dendrite vs P2 dendrite (2-coupled)')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('t (ms)')
axs[1].set_ylabel('v (mV)')
plt1.show()

In [ ]:
recording_cell = L1.sets[0].P1
recording_cell_2 = L1.sets[0].P2
axon_m1 = h.Vector().record(recording_cell.axon(0.5)._ref_v)
dend_m1 = h.Vector().record(recording_cell.dend(0.5)._ref_v)
axon_m2 = h.Vector().record(recording_cell_2.axon(0.5)._ref_v)
dend_m2 = h.Vector().record(recording_cell_2.dend(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)

fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(axon_m1), list(axon_m2), label='P1 axon vs P2 axon (2-coupled)')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('t (ms)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(dend_m1),list(dend_m2), label='P1 dendrite vs P2 dendrite (2-coupled)')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('t (ms)')
axs[1].set_ylabel('v (mV)')
plt1.show()

In [ ]:
recording_cell = L1.sets[0].G1
recording_cell_2 = L1.sets[0].G2
axon_m1 = h.Vector().record(recording_cell.axon(0.5)._ref_v)
dend_m1 = h.Vector().record(recording_cell.dend(0.5)._ref_v)
axon_m2 = h.Vector().record(recording_cell_2.axon(0.5)._ref_v)
dend_m2 = h.Vector().record(recording_cell_2.dend(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)
fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(axon_m1), list(axon_m2), label='G1 axon vs G2 axon (4-coupled)')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('t (ms)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(dend_m1),list(dend_m2), label='G1 dendrite vs G2 dendrite (4-coupled)')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('t (ms)')
axs[1].set_ylabel('v (mV)')
plt1.show()

In [ ]:
import numpy

OB_axon = []
OB_dend = []
AON_axon = []
AON_dend = []
for i in L1.sets:
    for j in enumerate(i.cells):
        if j[0] > 2:
            OB_axon.append(h.Vector().record(j[1].axon(0.5)._ref_v))
            OB_dend.append(h.Vector().record(j[1].dend(0.5)._ref_v))

for i in L1.AON.cells:
    AON_axon.append(h.Vector().record(i.axon(0.5)._ref_v))
    AON_dend.append(h.Vector().record(i.dend(0.5)._ref_v))
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)

OB_axon_plot = np.mean(OB_axon, 0)
OB_dend_plot = np.mean(OB_dend, 0)
AON_axon_plot = np.mean(AON_axon, 0)
AON_dend_plot = np.mean(AON_dend, 0)

fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(OB_axon_plot), list(AON_axon_plot), label='OB axon vs AON axon')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('t (ms)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(OB_dend_plot),list(AON_dend_plot), label='OB dendrite vs AON dendrite')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('t (ms)')
axs[1].set_ylabel('v (mV)')
plt1.show()

In [ ]:
import numpy

OB_axon = []
OB_dend = []
two_axon = []
two_dend = []
for i in L1.sets:
    for j in enumerate(i.cells):
        if j[0] > 2:
            OB_axon.append(h.Vector().record(j[1].axon(0.5)._ref_v))
            OB_dend.append(h.Vector().record(j[1].dend(0.5)._ref_v))

for i in L1.sets:
    for j in enumerate(i.cells):
        if j[0] < 2:
            two_axon.append(h.Vector().record(j[1].axon(0.5)._ref_v))
            two_dend.append(h.Vector().record(j[1].dend(0.5)._ref_v))
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)

OB_axon_plot = np.mean(OB_axon, 0)
OB_dend_plot = np.mean(OB_dend, 0)
two_axon_plot = np.mean(two_axon, 0)
two_dend_plot = np.mean(two_dend, 0)

fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(OB_axon_plot), list(two_axon_plot), label='OB axon vs 2-coupled axon')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('v (mV)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(OB_dend_plot),list(two_dend_plot), label='OB dendrite vs 2-coupled dendrite')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('v (mv)')
axs[1].set_ylabel('v (mV)')
plt1.show()

In [ ]:
import numpy

OB_axon = []
OB_dend = []
PC_axon = []
PC_dend = []
for i in L1.sets:
    for j in enumerate(i.cells):
        if j[0] > 2:
            OB_axon.append(h.Vector().record(j[1].axon(0.5)._ref_v))
            OB_dend.append(h.Vector().record(j[1].dend(0.5)._ref_v))

for i in L1.PC.cells:
    PC_axon.append(h.Vector().record(i.axon(0.5)._ref_v))
    PC_dend.append(h.Vector().record(i.dend(0.5)._ref_v))
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)

OB_axon_plot = np.mean(OB_axon, 0)
OB_dend_plot = np.mean(OB_dend, 0)
PC_axon_plot = np.mean(PC_axon, 0)
PC_dend_plot = np.mean(PC_dend, 0)

fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(OB_axon_plot), list(PC_axon_plot), label='OB axon vs PC axon')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('v (mV)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(OB_dend_plot),list(PC_dend_plot), label='OB dendrite vs PC dendrite')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('v (mV)')
axs[1].set_ylabel('v (mV)')
plt1.show()

# Scatter Plots (Before Learning)

In [ ]:
%matplotlib inline
h.finitialize(-70*mV)
h.continuerun(300*ms)
color=['green','orange','blue','red', 'pink', 'purple']
label=['P1', 'P2', 'M1','M2','G1','G2']
plt1.figure(figsize=(16,10))
for j in range(len(L1.sets)):
  #for i,cell in enumerate(L1.sets[j].cells):
    plt1.scatter(L1.sets[j].cells[0].spike_times, [0.5 + 0.7*j for i in range(len(L1.sets[j].cells[0].spike_times))], s=10,color=color[0]) # for P1
    plt1.scatter(L1.sets[j].cells[1].spike_times, [0.7 + 0.7*j for i in range(len(L1.sets[j].cells[1].spike_times))], s=10, color=color[1]) # for P2
    plt1.scatter(L1.sets[j].cells[2].spike_times, [0.9 + 0.7*j for i in range(len(L1.sets[j].cells[2].spike_times))], s=10, color=color[2]) # for M1
    plt1.scatter(L1.sets[j].cells[3].spike_times, [1.1 + 0.7*j for i in range(len(L1.sets[j].cells[3].spike_times))], s=10, color=color[3]) # for M2
    plt1.scatter(L1.sets[j].cells[4].spike_times, [1.3 + 0.7*j for i in range(len(L1.sets[j].cells[4].spike_times))], s=10, color=color[4]) # for G1
    plt1.scatter(L1.sets[j].cells[5].spike_times, [1.5 + 0.7*j for i in range(len(L1.sets[j].cells[5].spike_times))], s=10, color=color[5]) # for G2

plt1.xlabel('time(ms)')
plt1.legend(label)
plt1.show()

# Scratchpad

In [ ]:
# P1_axon_data = []
# P2_axon_data = []
# M1_axon_data = []
# M2_axon_data = []
# G1_axon_data = []
# G2_axon_data = []


# for i in range(len(L1.sets)):
#     P1_axon_data.append(h.Vector().record(L1.sets[i].P1.axon(0.5)._ref_v))
#     P2_axon_data.append(h.Vector().record(L1.sets[i].P2.axon(0.5)._ref_v))
#     M1_axon_data.append(h.Vector().record(L1.sets[i].M1.axon(0.5)._ref_v))
#     M2_axon_data.append(h.Vector().record(L1.sets[i].M2.axon(0.5)._ref_v))
#     G1_axon_data.append(h.Vector().record(L1.sets[i].G1.axon(0.5)._ref_v))
#     G2_axon_data.append(h.Vector().record(L1.sets[i].G2.axon(0.5)._ref_v))

# t = h.Vector().record(h._ref_t)
# h.finitialize(-70* mV)
# h.continuerun(300*ms)

# print(P1_axon_data)    
# arr = np.array(P1_axon_data)
# back_to_the_list = list(arr.sum(axis=0))
# f = plt2.figure(x_axis_label='time')
# f.line(P2_axon_data[0],P1_axon_data[0], line_width=1, line_color='black')
# plt2.show(f)

# Learning

In [ ]:
#@title Learning Mechanism { vertical-output: true }
import sys, time
tick = time.time()
weight_data = []

out = display(progress(0, epochs), display_id=True)

for l in range(epochs):
    #print("iteration no is",l)
    out.update(progress(l, epochs))    
    # print("\nmutual learning")
    P1_axon_data = []
    P1_dend_data = []
    P2_axon_data = []
    P2_dend_data = []
    M1_axon_data = []
    M1_dend_data = []
    M2_axon_data = []
    M2_dend_data = []
    G1_axon_data = []
    G1_dend_data = []
    G2_axon_data = []
    G2_dend_data = []
    P1_spike_times = []
    P2_spike_times = []
    M1_spike_times = []
    M2_spike_times = []
    G1_spike_times = []
    G2_spike_times = []

    h.finitialize(-70* mV)
    h.continuerun(300*ms)
    t = h.Vector().record(h._ref_t)
    
    for i in range(len(L1.sets)):
        P1_axon_data.append(list(h.Vector().record(L1.sets[i].P1.axon(0.5)._ref_v)))
        P1_dend_data.append(list(h.Vector().record(L1.sets[i].P1.dend(0.5)._ref_v)))
        P1_spike_times.append(list(L1.sets[i].P1.spike_times))
        P2_axon_data.append(list(h.Vector().record(L1.sets[i].P2.axon(0.5)._ref_v)))
        P2_spike_times.append(list(L1.sets[i].P2.spike_times))
        P2_dend_data.append(list(h.Vector().record(L1.sets[i].P2.dend(0.5)._ref_v)))
        M1_axon_data.append(list(h.Vector().record(L1.sets[i].M1.axon(0.5)._ref_v)))
        M1_dend_data.append(list(h.Vector().record(L1.sets[i].M1.dend(0.5)._ref_v)))
        M1_spike_times.append(list(L1.sets[i].M1.spike_times))
        M2_axon_data.append(list(h.Vector().record(L1.sets[i].M2.axon(0.5)._ref_v)))
        M2_dend_data.append(list(h.Vector().record(L1.sets[i].M2.dend(0.5)._ref_v)))
        M2_spike_times.append(list(L1.sets[i].M2.spike_times))
        G1_axon_data.append(list(h.Vector().record(L1.sets[i].G1.axon(0.5)._ref_v)))
        G1_dend_data.append(list(h.Vector().record(L1.sets[i].G1.dend(0.5)._ref_v)))
        G1_spike_times.append(list(L1.sets[i].G1.spike_times))
        G2_axon_data.append(list(h.Vector().record(L1.sets[i].G2.axon(0.5)._ref_v)))
        G2_dend_data.append(list(h.Vector().record(L1.sets[i].G2.dend(0.5)._ref_v)))
        G2_spike_times.append(list(L1.sets[i].G2.spike_times))

   
    # h.finitialize(-70 * mV)

    for i in range(len(L1.sets)):
        # 2-coupled STDP-based learning
        L1.sets[i].nc1.weight[0] += generate_L_weight_delta(P1_spike_times[i], P2_spike_times[i])
        # logging
        # print("P1_axon", P1_spike_times[i], P2_spike_times[i])
        L1.sets[i].nc2.weight[0] += generate_L_weight_delta(P2_spike_times[i], P1_spike_times[i])
        L1.sets[i].nc3.weight[0] += generate_L_weight_delta(P1_spike_times[i], M1_spike_times[i])
        # 4-coupled STDP-based learning
        L1.sets[i].nc4.weight[0] += generate_L_weight_delta(M1_spike_times[i], M2_spike_times[i])
        L1.sets[i].nc5.weight[0] += generate_L_weight_delta(M2_spike_times[i], M1_spike_times[i])
        L1.sets[i].nc6.weight[0] += generate_L_weight_delta(M2_spike_times[i], G1_spike_times[i])
        L1.sets[i].nc7.weight[0] += generate_L_weight_delta(G1_spike_times[i], M2_spike_times[i])
        L1.sets[i].nc8.weight[0] += generate_L_weight_delta(G1_spike_times[i], G2_spike_times[i])
        L1.sets[i].nc9.weight[0] += generate_L_weight_delta(G2_spike_times[i], G1_spike_times[i])
        L1.sets[i].nc10.weight[0] += generate_L_weight_delta(G2_spike_times[i], M1_spike_times[i])
        L1.sets[i].nc11.weight[0] += generate_L_weight_delta(M1_spike_times[i], G2_spike_times[i])
        L1.sets[i].nc12.weight[0] += generate_L_weight_delta(M1_spike_times[i], G1_spike_times[i])
        L1.sets[i].nc13.weight[0] += generate_L_weight_delta(G1_spike_times[i], M1_spike_times[i])
        # L1.sets[i].DPC.weight[0] += generate_L_weight_delta(G1_spike_times[i], M1_spike_times[i])
        
    # do a proper logging function
    weight_data.append(L1.sets[0].nc1.weight[0])


    #Lateral Learning
    for i in range(len(L1.sets)-1):
        # 2-coupled part
        L1.twoCupArr[i][0].weight[0] += generate_L_weight_delta(P1_spike_times[i], P1_spike_times[i+1])
        L1.twoCupArr[i][1].weight[0] += generate_L_weight_delta(P1_spike_times[i+1], P1_spike_times[i])
        # 4-coupled part
        L1.fourCupArr[i][0].weight[0] += generate_L_weight_delta(M1_spike_times[i], M1_spike_times[i+1])
        L1.fourCupArr[i][1].weight[0] += generate_L_weight_delta(M1_spike_times[i+1], M1_spike_times[i])
        L1.fourCupArr[i][2].weight[0] += generate_L_weight_delta(G1_spike_times[i], G1_spike_times[i+1])
        L1.fourCupArr[i][3].weight[0] += generate_L_weight_delta(G1_spike_times[i+1], G1_spike_times[i])
    # plotCell(L1.sets[-1].M1, 'M1')


out.update(progress(epochs, epochs))    
tock = time.time()
print("The time it took to learn", tock-tick, "seconds")

In [ ]:
#@title Time Taken { vertical-output: true }
totalend = time.time()
print("Total time it took: ", totalend-totalstart)

# Voltage vs Voltage Analysis (After Learning)

In [ ]:
import numpy

OB_axon = []
OB_dend = []
AON_axon = []
AON_dend = []
for i in L1.sets:
    for j in enumerate(i.cells):
        if j[0] > 2:
            OB_axon.append(h.Vector().record(j[1].axon(0.5)._ref_v))
            OB_dend.append(h.Vector().record(j[1].dend(0.5)._ref_v))

for i in L1.AON.cells:
    AON_axon.append(h.Vector().record(i.axon(0.5)._ref_v))
    AON_dend.append(h.Vector().record(i.dend(0.5)._ref_v))
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)

OB_axon_plot = np.mean(OB_axon, 0)
OB_dend_plot = np.mean(OB_dend, 0)
AON_axon_plot = np.mean(AON_axon, 0)
AON_dend_plot = np.mean(AON_dend, 0)

fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(OB_axon_plot), list(AON_axon_plot), label='OB axon vs AON axon')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('v (mV)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(OB_dend_plot),list(AON_dend_plot), label='OB dendrite vs AON dendrite')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('v (mV)')
axs[1].set_ylabel('v (mV)')
plt1.show()

In [ ]:
import numpy

OB_axon = []
OB_dend = []
two_axon = []
two_dend = []
for i in L1.sets:
    for j in enumerate(i.cells):
        if j[0] > 2:
            OB_axon.append(h.Vector().record(j[1].axon(0.5)._ref_v))
            OB_dend.append(h.Vector().record(j[1].dend(0.5)._ref_v))

for i in L1.sets:
    for j in enumerate(i.cells):
        if j[0] < 2:
            two_axon.append(h.Vector().record(j[1].axon(0.5)._ref_v))
            two_dend.append(h.Vector().record(j[1].dend(0.5)._ref_v))
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)

OB_axon_plot = np.mean(OB_axon, 0)
OB_dend_plot = np.mean(OB_dend, 0)
two_axon_plot = np.mean(two_axon, 0)
two_dend_plot = np.mean(two_dend, 0)

fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(OB_axon_plot), list(two_axon_plot), label='OB axon vs 2-coupled axon')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('v (mV)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(OB_dend_plot),list(two_dend_plot), label='OB dendrite vs 2-coupled dendrite')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('v (mV)')
axs[1].set_ylabel('v (mV)')
plt1.show()

In [ ]:
import numpy

OB_axon = []
OB_dend = []
PC_axon = []
PC_dend = []
for i in L1.sets:
    for j in enumerate(i.cells):
        if j[0] > 2:
            OB_axon.append(h.Vector().record(j[1].axon(0.5)._ref_v))
            OB_dend.append(h.Vector().record(j[1].dend(0.5)._ref_v))

for i in L1.PC.cells:
    PC_axon.append(h.Vector().record(i.axon(0.5)._ref_v))
    PC_dend.append(h.Vector().record(i.dend(0.5)._ref_v))
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)

OB_axon_plot = np.mean(OB_axon, 0)
OB_dend_plot = np.mean(OB_dend, 0)
PC_axon_plot = np.mean(PC_axon, 0)
PC_dend_plot = np.mean(PC_dend, 0)

fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(OB_axon_plot), list(PC_axon_plot), label='OB axon vs PC axon')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('v (mV)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(OB_dend_plot),list(PC_dend_plot), label='OB dendrite vs PC dendrite')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('v (mV)')
axs[1].set_ylabel('v (mV)')
plt1.show()

In [ ]:
recording_cell = L1.sets[-1].M1
recording_cell_2 = L1.AON.M1
axon_m1 = h.Vector().record(recording_cell.axon(0.5)._ref_v)
dend_m1 = h.Vector().record(recording_cell.dend(0.5)._ref_v)
axon_m2 = h.Vector().record(recording_cell_2.axon(0.5)._ref_v)
dend_m2 = h.Vector().record(recording_cell_2.dend(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(200 * ms)

# f = plt2.figure(x_axis_label='M1 Axon v (mV)', y_axis_label='AON Axon v (mV)')
# f.line(list(axon_m1), list(axon_m2), line_width=1,legend_label='4-coupled M1 axon vs AON axon',line_color='black')
# f2 = plt2.figure(x_axis_label='M1 Dendrite v (mV)', y_axis_label='AON dendrite v (mV)')
# f2.line(list(dend_m1), list(dend_m2), line_width=2,legend_label='4-coupled M1 dendrite vs AON dendrite',line_color='black')
# plt2.show(f)
# plt2.show(f2)

fig, axs = plt1.subplots(2, figsize=(16,18))
axs[0].plot(list(axon_m1), list(axon_m2), label='4-coupled M1 axon vs AON axon')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('v (mV)')
axs[0].set_ylabel('v (mV)')
axs[1].plot(list(dend_m1),list(dend_m2), label='4-coupled M1 dendrite vs AON dendrite')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('v (mV)')
axs[1].set_ylabel('v (mV)')
plt1.show()

# Weights (While Learning)

In [ ]:
print(weight_data)
f = plt1.figure()
x = [i for i in range(epochs)]
# f.line(x, weight_data, line_width=1, line_color='black')
plt1.plot(x, weight_data)
plt1.show(f)

# Voltages (After Learning)

In [ ]:
plotCell(L1.sets[-1].P1, 'P1')

In [ ]:
plotCell(L1.sets[-1].P2, 'P2')

In [ ]:
plotCell(L1.sets[-1].M1, 'M1')

In [ ]:
plotCell(L1.sets[-1].G1, 'G1')

In [ ]:
plotCell(L1.sets[-1].M2, 'M2')

In [ ]:
plotCell(L1.sets[-1].G2, 'G2')

In [ ]:
plotCell(L1.DPC, 'DPC')

In [ ]:
plotCell(L1.AON.M1, 'AON')

In [ ]:
plotCell(L1.PC.G1, 'Pyramidal Cell')

# Scatter Plots (After Learning)

In [ ]:
%matplotlib inline
h.finitialize(-70*mV)
h.continuerun(300*ms)
color=['green','orange','blue','red', 'pink', 'purple']
label=['P1', 'P2', 'M1','M2','G1','G2']
plt1.figure(figsize=(16,10))
for j in range(len(L1.sets)):
  #for i,cell in enumerate(L1.sets[j].cells):
    plt1.scatter(L1.sets[j].cells[0].spike_times, [0.5 + 0.7*j for i in range(len(L1.sets[j].cells[0].spike_times))], s=10,color=color[0]) # for P1
    plt1.scatter(L1.sets[j].cells[1].spike_times, [0.7 + 0.7*j for i in range(len(L1.sets[j].cells[1].spike_times))], s=10, color=color[1]) # for P2
    plt1.scatter(L1.sets[j].cells[2].spike_times, [0.9 + 0.7*j for i in range(len(L1.sets[j].cells[2].spike_times))], s=10, color=color[2]) # for M1
    plt1.scatter(L1.sets[j].cells[3].spike_times, [1.1 + 0.7*j for i in range(len(L1.sets[j].cells[3].spike_times))], s=10, color=color[3]) # for M2
    plt1.scatter(L1.sets[j].cells[4].spike_times, [1.3 + 0.7*j for i in range(len(L1.sets[j].cells[4].spike_times))], s=10, color=color[4]) # for G1
    plt1.scatter(L1.sets[j].cells[5].spike_times, [1.5 + 0.7*j for i in range(len(L1.sets[j].cells[5].spike_times))], s=10, color=color[5]) # for G2

plt1.xlabel('time(ms)')
plt1.legend(label)
plt1.show()

# Scratchpad 

In [ ]:
name = 'DPC'
addendum=""
recording_cell = L1.DPC
cell_axon = h.Vector().record(recording_cell.axon(0.5)._ref_v)
cell_dend = h.Vector().record(recording_cell.dend(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

h.finitialize(-70 * mV)
h.continuerun(300 * ms)

f = plt2.figure(x_axis_label='t (ms)', y_axis_label='v (mV)')
f.line(t, list(cell_axon), line_width=1,legend_label=name+' axon '+addendum,line_color='black')
f.line(t, list(cell_dend), line_width=2,legend_label=name+' dendrite '+addendum,line_color='red', line_dash='dashed')
plt2.show(f)